# Our Quantum Simulator
## Group: Brandon, Chris, Jacob, Michael

In [6]:
import gates
import utilities

TypeError: 'type' object is not subscriptable

In [7]:
class QuantumProgram():
    def __init__(self,the_number_of_qubits):
        self.qubits = gates.zeroQubit
        self.qubit_number = the_number_of_qubits
        self.operations = []
        self.classical = []
        for i in range(1, the_number_of_qubits):
            self.qubits = utilities.tensor(self.qubits,gates.zeroQubit)
    def getQubits(self):
        return self.qubits
    def addQubits(self, the_number_of_qubits):
        self.qubit_number += the_number_of_qubits
        for i in range(0, the_number_of_qubits):
            self.qubits = utilities.tensor(self.qubits,gates.zeroQubit)
    def getOperations(self):
        return self.operations
    def getNumberOfOperations(self):
        return len(self.operations)
    
    def getQubit(self,position):
        return [self.qubits[position*2],self.qubits[position*2+1]]
    def getClassical(self, position):
        return self.classical[position]
    def measureQubit(self,qubitPosition,classicalRegister=None):
        qubit = self.getQubit(qubitPosition)
        zeroRange = qubit[0] * 100
        random.seed()
        measurement = int(random()*100)
        if measurement <= zeroRange:
            measurement = 0
        else:
            measurement = 1
        if classicalRegister == None:
            self.classical.append(measurement)
        else:
            self.classical[classicalRegister] = measurement
        return measurement
    def measureAll(self):
        self.classical = []
        for i in range(0, self.qubit_number):
            measureQubit(i)
        return self.classical
    def measureAllAsString(self):
        self.measureAll()
        outString = ""
        for bit in self.classical:
            outString += chr(bit)
        return outString
        
    def H(self, position_of_qubit):
        (self.qubits,returnedGate) = gates.apply_gate(gates.H(),self.qubits,position_of_qubit)
        self.operations.append(returnedGate)
    def NOT(self, position_of_qubit):
        (self.qubits,returnedGate) = gates.apply_gate(gates.NOT(),self.qubits,position_of_qubit)
        self.operations.append(returnedGate)
    def Z(self, position_of_qubit):
        (self.qubits,returnedGate) = gates.apply_gate(gates.Z(),self.qubits,position_of_qubit)
        self.operations.append(returnedGate)
    def R(self, position_of_qubit,theta:float):
        (self.qubits,returnedGate) = gates.apply_gate(gates.R(theta),self.qubits,position_of_qubit)
        self.operations.append(returnedGate)
    def Control(gateFunction,position_of_qubit,controls=[],theta=None):#Gate function being one of the above gates
        mat2=[]
        gateReturn = []
        if theta != None:#Not sure if overriding the given gateFunction with the one from gates.py works, but everything else should.
            mat2,gateReturn=gates.applyGate(gates.gateFunction(theta),self.qubits,position_of_qubit,controls)
        else: 
            mat2,gateReturn=gates.applyGate(gates.gateFunction(),self.qubits,position_of_qubit,controls)
        self.qubits = mat2
        self.operations.append(gateReturn)
        return mat2
    def ControlClassic(gateFunction,position_of_qubit,controls=[],theta=None):#Gate function being one of the above gates
        mat2=[]
        gateReturn = []
        if 0 not in controls:
            if theta != None:#Not sure if overriding the given gateFunction with the one from gates.py works, but everything else should.
                mat2,gateReturn=gates.applyGate(gates.gateFunction(theta),self.qubits,position_of_qubit,controls)
            else: 
                mat2,gateReturn=gates.applyGate(gates.gateFunction(),self.qubits,position_of_qubit)
        else:
            mat2,gateReturn=gates.applyGate(gates.I(),self.qubits,position_of_qubit)
        self.qubits = mat2
        self.operations.append(gateReturn)
        return mat2
    
    def read_unitary():
        resultant = self.operations[0]
        first = True
        for operation in self.operations:
            if first:
                first = False
                continue
            else:
                resultant = utilities.matmul(resultant,operation)
        return resultant
    def read_state():
        return self.qubits
    def observing_probabilities():
        probabilities = []
        for i in range(0, self.qubit_number):
            probabilities.append(getQubit(i))
        return probabilities
    def execute(the_number_of_shots):
        executionDictionary = {}
        
        for string in utilities.getBinaryStrings(self.qubit_number):
            executionDictionary.setdefault(string,0)
        
        for i in range(0,the_number_of_shots):
            executionDictionary[self.measureAllAsString()] += 1
            
        return executionDictionary
            
        

In [8]:
qc = QuantumProgram(3)
qc.H(1)
qc.Control(NOT,2,[1])
qc.Control(NOT,1,[0])
qc.H(0)

qc.measureQubit(1)
qc.measureQubit(0)

qc.ControlClassic(NOT,2,[qc.getClassical(0)])
qc.ControlClassic(Z,2,[qc.getClassical(1)])

system = qc.getQubits()
print(system)

results = qc.execute(50)
print(results)


NameError: name 'gates' is not defined